<a href="https://colab.research.google.com/github/PsorTheDoctor/kaggle/blob/master/NLP/2_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Klasyfikacja tekstu

In [3]:
import pandas as pd

# Załadowanie danych
# ham to etykieta dla wiadomości, które nie są spamem
spam = pd.read_csv('spam.csv')
spam.head(10)

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


### Budowanie modelu Bag of Words

In [0]:
import spacy

# Stworzenie pustego modelu
nlp = spacy.blank('en')

# Stworzenie klasyfikatora tekstu z wyłącznymi klasami i architekturą "bow"
textcat = nlp.create_pipe(
    'textcat',
    config={
        'exclusive_classes': True,
        'architecture': 'bow'})

# Dodanie klasyfikatora tekstu do pustego modelu
nlp.add_pipe(textcat)

In [6]:
# Dodanie etykiet do klasyfikatora tekstu 
textcat.add_label('ham')
textcat.add_label('spam')

1

### Trenowanie modelu klasyfikatora tekstu

In [0]:
train_texts = spam['text'].values
train_labels = [{'cats': {'ham': label == 'ham',
                          'spam': label == 'spam'}}
                for label in spam['label']]

In [10]:
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  {'cats': {'ham': True, 'spam': False}}),
 ('Ok lar... Joking wif u oni...', {'cats': {'ham': True, 'spam': False}}),
 ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  {'cats': {'ham': False, 'spam': True}})]

In [0]:
from spacy.util import minibatch

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

# Stworzenie generatora wsadów z rozmiarem wsadu (batch size = 8)
batches = minibatch(train_data, size=8)
# Iteracaja po miniwsadach
for batch in batches:
  # Każdy wsad jest listą (tekst, etykieta), ale potrzebujemy
  # wysłać osobne listy dla tekstów i etykiet do update()
  # To jest szybki sposób listy tupli na listy
  text, labels = zip(*batch)
  nlp.update(text, labels, sgd=optimizer)

In [13]:
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}

for epoch in range(10):
  random.shuffle(train_data)
  # Stworzenie generatora wsadów z rozmiarem wsadu = 8
  batches = minibatch(train_data, size=8)
  # Iteracaja po miniwsadach
  for batch in batches:
    # Każdy wsad jest listą (tekst, etykieta), ale potrzebujemy
    # wysłać osobne listy dla tekstów i etykiet do update()
    # To jest szybki sposób listy tupli na listy
    text, labels = zip(*batch)
    nlp.update(text, labels, sgd=optimizer, losses=losses)
  print(losses)

{'textcat': 0.29890368430487513}
{'textcat': 0.4806387926536857}
{'textcat': 0.5982820415381982}
{'textcat': 0.672603283121775}
{'textcat': 0.7211023606003482}
{'textcat': 0.7543036989518874}
{'textcat': 0.7799389380317288}
{'textcat': 0.7962947575560354}
{'textcat': 0.8090532684510547}
{'textcat': 0.8178490321991748}


### Tworzenie predykcji

In [14]:
texts = ["Are you ready for a tea party????? It's gonna to be wild",
         "URGENT Reply for this message for GUARANTEED FREE TEA"]
docs = [nlp.tokenizer(text) for text in texts]

# Użycie textcata do dostania się do wyników każdego doca
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

[[9.998926e-01 1.073430e-04]
 [5.668400e-03 9.943316e-01]]


In [17]:
# Znalezienie etykiety z najwyższym wynikiem/prawdopodobieństwem ze scores 
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['ham', 'spam']


### Źródło:
[https://www.kaggle.com/matleonard/text-classification](https://www.kaggle.com/matleonard/text-classification)